<a href="https://colab.research.google.com/github/marco-siino/SemEval2024/blob/main/Task%2010/SemEval_2024_Task_10_Mistral7B_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [2]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets
!pip install -U deep-translator

import datasets
from datasets import load_dataset
from deep_translator import GoogleTranslator
import json
import re
import random
import numpy as np
import tqdm.notebook as tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 59.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 81.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 239.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 337.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.38-cp310-cp310-manylinux_2_35_x86_64.whl size=9697774 sha256=66f7aa0443e03ca9c4b2249b9e2f199e48408f7c58db4a36f20fb12e7a0c25cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-ze8r8m0s/wheels/eb/58/77/20d3d9a235b4930050fbcde1ad4f0a4d054644269e801b08aa
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.9.0
    Uninstalling typing_extensions-4.9.

Downloading an instruction-finetuned Mistral model.

In [3]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4, #16, # CPU cores
    n_batch=800, #8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '1000000.000000', 'llama.context_length': '32768', 'general.na

# Download the dataset for the three subtasks.

In [4]:
# Subtask 1.

!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_train_erc.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_val_erc.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_test_erc.json'

train_path_1 = 'MaSaC_train_erc.json'
val_path_1 = 'MaSaC_val_erc.json'
test_path_1 = 'MaSaC_test_erc.json'

--2024-02-03 15:27:13--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_train_erc.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 720262 (703K) [text/plain]
Saving to: ‘MaSaC_train_erc.json.2’

MaSaC_train_erc.jso 100%[===================>] 703.38K  --.-KB/s    in 0.04s   

2024-02-03 15:27:14 (15.7 MB/s) - ‘MaSaC_train_erc.json.2’ saved [720262/720262]

--2024-02-03 15:27:14--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_val_erc.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected

In [5]:
# Subtask 2.

!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_train_efr.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_val_efr.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_test_efr.json'

train_path_2 = 'MaSaC_train_efr.json'
val_path_2 = 'MaSaC_val_efr.json'
test_path_2 = 'MaSaC_test_efr.json'

--2024-02-03 15:27:14--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_train_efr.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8644822 (8.2M) [text/plain]
Saving to: ‘MaSaC_train_efr.json.2’

MaSaC_train_efr.jso 100%[===================>]   8.24M  --.-KB/s    in 0.09s   

2024-02-03 15:27:14 (92.2 MB/s) - ‘MaSaC_train_efr.json.2’ saved [8644822/8644822]

--2024-02-03 15:27:15--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_val_efr.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connec

In [6]:
# Subtask 3.

!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_train_efr.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_val_efr.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_test_efr.json'

train_path_3 = 'MELD_train_efr.json'
val_path_3 = 'MELD_val_efr.json'
test_path_3 = 'MELD_test_efr.json'

--2024-02-03 15:27:15--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_train_efr.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5180775 (4.9M) [text/plain]
Saving to: ‘MELD_train_efr.json.2’

MELD_train_efr.json 100%[===================>]   4.94M  --.-KB/s    in 0.07s   

2024-02-03 15:27:16 (68.4 MB/s) - ‘MELD_train_efr.json.2’ saved [5180775/5180775]

--2024-02-03 15:27:16--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_val_efr.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.

# Create few-shot samples from training and validation set.

In [7]:
def create_few_shot_samples_task1(json_set_path,nr_samples):

  nr_few_shot_samples = nr_samples

  few_shot_counter = 0

  few_shot_samples = ''

  with open(json_set_path, 'r') as istr:
    train_1_json = json.load(istr)
  for i in tqdm.trange(len(train_1_json)):
    if few_shot_counter == nr_few_shot_samples:
      break

    for sentence_nr in range(0,len(train_1_json[i]['speakers'])):
      few_shot_counter+=1
      #print(train_1_json[i]['speakers'][sentence_nr])
      translated_utterance = GoogleTranslator(source='hi', target='en').translate(train_1_json[i]['utterances'][sentence_nr])
      few_shot_samples += train_1_json[i]['speakers'][sentence_nr]+' // ' + translated_utterance + ' // ' + train_1_json[i]['emotions'][sentence_nr] + " \n "

      if few_shot_counter == nr_few_shot_samples:
        break

  return few_shot_samples

In [26]:
few_shot_samples = create_few_shot_samples_task1(train_path_1,20)
few_shot_samples += create_few_shot_samples_task1(val_path_1,20)

  0%|          | 0/343 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

In [18]:
print(few_shot_samples)

maya // What does Indravadan keep in the house? Indravadan please why didn't you throw away all the useless things? // disgust 
 indu // ok, let's rise, let's spring // contempt 
 rosesh // Momma! Hath Chodiye Dad! // anger 
 indu // Look Maya or I am not speaking! Then you will say that you are not throwing the wrong things, come on. // neutral 
 maya // Indravadan, for how many years have you been telling this joke? What is happening today? // neutral 
 indu // joke on kaikai // contempt 
 rosesh // very funny // contempt 
 indu // Maya, Monisha and Sahil are here, see what they are using and see how much they have done. // neutral 
 sahil // hi guys // joy 
 rosesh // hi // joy 
 good rosesh // What are you doing Rosesh? India is your country! Bhartiya is a true human being. It is better not to read anyone's letter. // neutral 
 bad rosesh // hahaha read the book rosesh baby or read the book also. How nice looks the gel in the balloon, ding dong bell. // neutral 
 good rosesh // Poe

# Collect all the emotions in the train and in the dev set for subtask 1.

In [23]:
emotions_list = []

# Train set first.
with open(train_path_1, 'r') as istr:
  file_json = json.load(istr)
  for i in tqdm.trange(len(file_json)):
    for sentence_nr in range(0,len(file_json[i]['emotions'])):
      #print(file_json[i]['emotions'][sentence_nr])
      if file_json[i]['emotions'][sentence_nr] not in emotions_list:
        emotions_list.append(file_json[i]['emotions'][sentence_nr])
      break

# Validation set after.
with open(val_path_1, 'r') as istr:
  file_json = json.load(istr)
  for i in tqdm.trange(len(file_json)):
    for sentence_nr in range(0,len(file_json[i]['emotions'])):
      #print(file_json[i]['emotions'][sentence_nr])
      if file_json[i]['emotions'][sentence_nr] not in emotions_list:
        emotions_list.append(file_json[i]['emotions'][sentence_nr])
      break



  0%|          | 0/343 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

In [24]:
emotions_list

['disgust',
 'joy',
 'neutral',
 'anger',
 'sadness',
 'contempt',
 'surprise',
 'fear']

# Run!

In [27]:
with open('answer.txt', 'w') as f:
        f.write('')

# simple JSON loading
with open(test_path_1, 'r') as istr:
    data_val_all = json.load(istr)
num_sample = len(data_val_all)
#print(num_sample)


with open(test_path_1, 'r') as istr:
    test_1_json = json.load(istr)
for i in range(0,len(test_1_json)):
  current_sample =''
  for sentence_nr in range(0,len(test_1_json[i]['speakers'])):
      print("\n\n")
      print("ORIGINAL: "+ test_1_json[i]['speakers'][sentence_nr]+' // '+test_1_json[i]['utterances'][sentence_nr])

      try:
        translated_utterance = GoogleTranslator(source='hi', target='en').translate(test_1_json[i]['utterances'][sentence_nr])
      except:
        translated_utterance = test_1_json[i]['utterances'][sentence_nr]

      current_sample = test_1_json[i]['speakers'][sentence_nr]+' // ' + translated_utterance + ' // '

      prompt = "\n Use the CONTEXT to complete the SENTENCE using ONLY one emotion among: disgust, joy, neutral, anger, sadness, contempt, surprise, fear. Do not explain!\n"
      prompt += f"<s>[INST] CONTEXT: {few_shot_samples}\n SENTENCE:{current_sample} [/INST]"

      #print(prompt)

      response = lcpp_llm(
        prompt=prompt,
        temperature= 0.5,
        logprobs=1,
      )

      answer = str(response["choices"][0]["text"]).strip().lower()
      answer = answer.split()[0]
      # Sometime output contains a '.' remove it!
      answer = answer.replace('.','')

      # If the predicted word is not in emotion list just replace with neutral.
      if answer not in emotions_list:
        answer = 'neutral'

      #current_sample += answer + " \n "

      print("GENERATED: "+ current_sample+answer)

      with open('answer.txt', 'a') as f:
        f.write(answer+"\n")

break





ORIGINAL: maya // kuchh karo sahil please kuchh karo. mera rosesh adopt ho karke chala gaya na to me, i know this sounds horribly melodramatic, monishaish, par me mar jaaungi. i swear mein mar jaaungi


Llama.generate: prefix-match hit


GENERATED: maya // Please go to the beach. My rosesh started adopting me, I know this sounds horribly melodramatic, monishaish, but I will die. i swear i will live in the sea // fear



ORIGINAL: indu // monisha tum muskura kya rahi ho?


Llama.generate: prefix-match hit


GENERATED: indu // monisha why are you smiling? // neutral



ORIGINAL: monisha // haila! main? main kahan? kya daddy ji?


...

GENERATED: madhu // hain? // neutral



ORIGINAL: rosesh // inko sunai nahin deta


Llama.generate: prefix-match hit


GENERATED: rosesh // I did not hear anything. // neutral



ORIGINAL: theif // are yaar sunai nahin deta lekin dikhai to deta hai. aa gale mein chaku rakha dikhai nahin deta


Llama.generate: prefix-match hit


KeyboardInterrupt: 

In [ ]:
!zip -r results.zip result.json

  adding: result.json (deflated 94%)


In [ ]:
from google.colab import files
files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>